# Agenda

1. Descriptors (vs. properties)
2. Iterators
     - Iterators and classes
     - Generator functions
     - Generator expressions
3. Decorators
4. Concurrency
    - Threads
    - Processes
    - Executors

In [2]:
d = {'a':1, 'b':2, 'c':3}

In [3]:
mylist = [100, 200, 300]
d[mylist] = 4

TypeError: unhashable type: 'list'

In [4]:
t = (10, 20, 30)     # tuple is immutable, thus hashable
d[t] = 100

In [6]:
t = (10, 20, [100, 200, 300])  # tuple is immutable, *BUT* contains mutable, so *NOT* hashable
d[t] = 100

TypeError: unhashable type: 'list'

In [7]:
# mutable and hashable? YES

class Person:
    def __init__(self, name):
        self.name = name
        
p = Person('Reuven')    

d[p] = 100

In [8]:
p.name

'Reuven'

In [9]:
p.name = 'something else'

In [10]:
d[p]

100

In [11]:
hash(p)

284314249

In [12]:
id(p)

4549027984

In [13]:
class Person:
    def __init__(self, name):
        self.name = name
        
    def __hash__(self):
        return hash(self.name)
        
p = Person('Reuven')    

d[p] = 100

In [14]:
d[p]

100

In [15]:
p.name = 'something else'

In [16]:
d[p]

KeyError: <__main__.Person object at 0x10f24ab30>

In [17]:
import random

class Person:
    def __init__(self, name):
        self.name = name
        
    def __hash__(self):
        return random.randint(0, 100_000)
        
p = Person('Reuven')    
d[p] = 100

In [18]:
d[p]

KeyError: <__main__.Person object at 0x10f24ae30>

In [19]:
# Properties
# looks like data, acts like getter/setter

class Thermostat:
    def __init__(self):
        self._temp = 20
        
    @property
    def temp(self):
        return self._temp
    
    @temp.setter
    def temp(self, new_temp):
        if new_temp < 10:
            raise ValueError('Too low!')
        
        if new_temp > 40:
            raise ValueError('Too high!')
        
        self._temp = new_temp

In [20]:
t = Thermostat()

t.temp

20

In [21]:
t.temp = 25
t.temp

25

In [22]:
t.temp = -10

ValueError: Too low!

In [24]:
type(t.temp)  # request the attribute temp from t, an instance

int

In [25]:
type(Thermostat.temp)  # request the attribute temp from Thermostat, the class

property

# Descriptor

Properties implement the descriptor protocol:

- If I have a class attribute,
- and I request its via the instance,
- and if the object in the class attribute has the `__get__` method,
- then the `__get__` method runs, and we get its value.

In [30]:
# Properties
# looks like data, acts like getter/setter

class SafeTemp:
    def __init__(self):
        self._temp = 20
        
    def __get__(self, instance, owner_class):
        print(f'Called SafeTemp.__get__: {instance=}, {owner_class=}')
        return self._temp
    
    def __set__(self, instance, new_value):
        print(f'Called SafeTemp.__set__: {instance=}, {new_value=}')
        self._temp = new_value

class Thermostat:
    temp = SafeTemp()   # class attribute, instance of SafeTemp
    
t = Thermostat()
t.temp  # class attribute, ask for the value via the instance
t.temp = 25   # 

Called SafeTemp.__get__: instance=<__main__.Thermostat object at 0x11b18bd30>, owner_class=<class '__main__.Thermostat'>
Called SafeTemp.__set__: instance=<__main__.Thermostat object at 0x11b18bd30>, new_value=25


In [31]:
t.temp

Called SafeTemp.__get__: instance=<__main__.Thermostat object at 0x11b18bd30>, owner_class=<class '__main__.Thermostat'>


25

In [32]:
t.temp = 20

Called SafeTemp.__set__: instance=<__main__.Thermostat object at 0x11b18bd30>, new_value=20


In [33]:
class MyClass:
    def __init__(self, x):
        self.x = x
        
    def x2(self):
        return self.x * 2
    
m = MyClass(10)
m.x2()

20

In [34]:
m.x2

<bound method MyClass.x2 of <__main__.MyClass object at 0x11b18af20>>

In [ ]:
m.x2()  #  --->  MyClass.x2(m)

In [35]:
MyClass.__dict__

mappingproxy({'__module__': '__main__',
              '__init__': <function __main__.MyClass.__init__(self, x)>,
              'x2': <function __main__.MyClass.x2(self)>,
              '__dict__': <attribute '__dict__' of 'MyClass' objects>,
              '__weakref__': <attribute '__weakref__' of 'MyClass' objects>,
              '__doc__': None})

In [36]:
class MyClass:
    def __init__(self, x):
        self.x = x
        
    @staticmethod
    def hello():
        return 'Hello!'
    
m = MyClass(10)
m.hello()

'Hello!'

In [37]:
MyClass.hello()

'Hello!'

In [38]:
type(MyClass.hello)

function

In [39]:
type(m.hello)

function

# Iterators

In [40]:
for one_item in 'abcd':
    print(one_item)

a
b
c
d


In [41]:
for one_item in 5:
    print(one_item)

TypeError: 'int' object is not iterable

In [42]:
s = 'abcd'

# ask if the object is iterable
iter('abcd')

In [43]:
iter('abcd')

In [44]:
iter('abcd')

In [45]:
# if it is iterable, store the iterator object in i
i = iter('abcd')

In [46]:
# ask for the next item
next(i)

'a'

In [47]:
next(i)

'b'

In [48]:
next(i)

'c'

In [49]:
next(i)

'd'

In [50]:
next(i)

StopIteration: 

1. Ask for an object's iterator with `iter`
   - If not iterable, we get a `TypeError` exception
   - If iterable, we get an iterator object back
2. Run `next` on that iterator that we got back
   - If we get a `StopIteration` exception, we stop
   - Or we get an object back, the next value
3. Return to step 2   